In [1]:
# baseline model with dropout on the cifar10 dataset
import sys
from matplotlib import pyplot
from keras.datasets import cifar10
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from tensorflow.keras.optimizers import SGD

# Model evaluation test harness

**Load dataset**

We know that all the images have:
- Same square size of 32x32 pixels
- Labeled and divided into 10 classes
- The images are color

Therefore we can load the images and use them for modeling almost immediately.

In [10]:
# load train and test dataset
def load_dataset():
	# load dataset
	(trainX, trainY), (testX, testY) = cifar10.load_data()
 
	# one hot encode target values
	trainY = np_utils.to_categorical(trainY)
	testY = np_utils.to_categorical(testY)
 
	return trainX, trainY, testX, testY

Pixel values for each image in the dataset are unsigned integers in the range between no color and full color (0 and 255). We need to scale the pixel values for modeling, but we don't know how exactly. To solve this problem, we can normalize the pixel values, rescale them to the range [0, 1], and because our data is intergers, so first we need to convert them into floats, then divide the pixel values by the maximum value.

In [7]:
# scale pixels
def prep_pixels(train, test):
	train_norm = train.astype('float32')
	test_norm = test.astype('float32')
 
	# normalize to range 0-1
	train_norm = train_norm / 255.0
	test_norm = test_norm / 255.0
	
	return train_norm, test_norm

Next step, we define our model. This step we will discuss later because it requires some knowledge and would take a little bit of time. 
Once the model has been evaluated, we can present the results:
- The diagnostics of the learning behavior of the model during training, by creating a line spot showing model performance on the train and test set during training. These plots are valuable for getting an idea of whether a model is overfitting, underfitting, or has a good fit for the dataset.
- The estimation of the model performance. We will create a single figure with two subplots, one for loss and one for accuracy. The blue lines will indicate model performance on the training dataset and orange lines will indicate performance on the hold out test dataset.

In [5]:
# plot diagnostic learning curves
def summarize_diagnostics(history):
	# plot loss
	pyplot.subplot(211)
	pyplot.title('Cross Entropy Loss')
	pyplot.plot(history.history['loss'], color='blue', label='train')
	pyplot.plot(history.history['val_loss'], color='orange', label='test')
 
	# plot accuracy
	pyplot.subplot(212)
	pyplot.title('Classification Accuracy')
	pyplot.plot(history.history['accuracy'], color='blue', label='train')
	pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
 
	# save plot to file
	filename = sys.argv[0].split('/')[-1]
	pyplot.savefig(filename + '_plot.png')
	pyplot.close()

Now we implement the test harness to kick-off the evaluation of a given model.

In [15]:
# run the test harness for evaluating a model
def run_test_harness():
	# load dataset
	trainX, trainY, testX, testY = load_dataset()
 
	# prepare pixel data
	trainX, testX = prep_pixels(trainX, testX)
 
	# define model
	print('Defining machine learning model...')
	model = define_model()
 
	# fit model
	print('Fitting machine learning model...')
	history = model.fit(trainX, trainY, epochs=100, batch_size=64, validation_data=(testX, testY), verbose=0)
	
	# evaluate model
	print('Evaluating machine learning model...')
	_, acc = model.evaluate(testX, testY, verbose=0)
 
  # Report final model performance on the test dataset
	print('> %.3f' % (acc * 100.0))
	summarize_diagnostics(history)

# Develop a baseline model

Here we use VGG architecture with 3 blocks, for details:
- The first block has 32 3x3 filters. This quantity should double after each block. We have 3 blocks, so the number of filters in each block should be 32, 63, 128 respectively with 'he_uniform' kernal_initializer.
- Each block has 2 convolution layers with same padding and followed by ReLU activation function.
- Each block has a 2x2 max pooling layer.

After 3 convolution layers, we flatten the tensor output and pass it into a fully-connected layer with ReLU activation function. Finally, we use softmax function to get the prediction output.


In [14]:
# define baseline cnn model
def define_model():
	model = Sequential()
 
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3)))
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(MaxPooling2D((2, 2)))
 
	model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(MaxPooling2D((2, 2)))
 
	model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(MaxPooling2D((2, 2)))
 
	model.add(Flatten())
	model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
	model.add(Dense(10, activation='softmax'))
 
	# compile model
	opt = SGD(lr=0.001, momentum=0.9)
	model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
	return model

Let's try our model!

In [ ]:
run_test_harness()

Defining machine learning model...
Fitting machine learning model...


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)
